# Using Interval Variables and AddNoOverlap2D constraint for 2D rectangle packing

Given a rectangular container and a set of rectangular items, the goal is to pack all items into the container respecting the orientation of items.
All rectangles are orthogonal, i.e. they are parallel to the axes of the container.
Thus, each item is defined by its length and width.

This problem is also known as the 2D bin packing problem and is NP-hard.
With CP-SAT, we can easily model this problem and solve smaller instances to optimality.

In [1]:
from ortools.sat.python import cp_model

In [2]:
# Instance
container = (40, 15)
boxes = [
    (11, 3),
    (13, 3),
    (9, 2),
    (7, 2),
    (9, 3),
    (7, 3),
    (11, 2),
    (13, 2),
    (11, 4),
    (13, 4),
    (3, 5),
    (11, 2),
    (2, 2),
    (11, 3),
    (2, 3),
    (5, 4),
    (6, 4),
    (12, 2),
    (1, 2),
    (3, 5),
    (13, 5),
    (12, 4),
    (1, 4),
    (5, 2),
    # (6,  2),  # add to make tight
    # (6,3), # add to make infeasible
]

In [3]:
def pack(container, boxes, scale: int):
    container = (container[0] * scale, container[1] * scale)
    boxes = [(box[0] * scale, box[1] * scale) for box in boxes]
    model = cp_model.CpModel()
    # We have to create the variable for the bottom left corner of the boxes.
    # We directly limit their range, such that the boxes are inside the container
    x_vars = [
        model.NewIntVar(0, container[0] - box[0], name=f"x1_{i}")
        for i, box in enumerate(boxes)
    ]
    y_vars = [
        model.NewIntVar(0, container[1] - box[1], name=f"y1_{i}")
        for i, box in enumerate(boxes)
    ]
    # Interval variables are actually more like constraint containers, that are then passed to the no overlap constraint
    # Note that we could also make size and end variables, but we don't need them here
    x_interval_vars = [
        model.NewIntervalVar(
            start=x_vars[i], size=box[0], end=x_vars[i] + box[0], name=f"x_interval_{i}"
        )
        for i, box in enumerate(boxes)
    ]
    y_interval_vars = [
        model.NewIntervalVar(
            start=y_vars[i], size=box[1], end=y_vars[i] + box[1], name=f"y_interval_{i}"
        )
        for i, box in enumerate(boxes)
    ]
    # Enforce that no two rectangles overlap
    model.AddNoOverlap2D(x_interval_vars, y_interval_vars)
    # Solve!
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    solver.log_callback = print
    status = solver.Solve(model)
    assert status == cp_model.OPTIMAL

In [4]:
pack(container, boxes, 1)



Starting CP-SAT solver v9.8.3296
Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Parameters: log_search_progress: true
Setting number of workers to 16
Setting number of workers to 16


Initial satisfaction model '': (model_fingerprint: 0x932179154af0ad79)
#Variables: 48
  - 3 in [0,10]
  - 6 in [0,11]
  - 6 in [0,12]
  - 9 in [0,13]
  - 4 in [0,27]
  - 2 in [0,28]
  - 5 in [0,29]
  - 2 in [0,31]
  - 2 in [0,33]
  - 1 in [0,34]
  - 2 in [0,35]
  - 2 in [0,37]
  - 2 in [0,38]
  - 2 in [0,39]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Initial satisfaction model '': (model_fingerprint: 0x932179154af0ad79)
#Variables: 48
  - 3 in [0,10]
  - 6 in [0,11]
  - 6 in [0,12]
  - 9 in [0,13]
  - 4 in [0,27]
  - 2 in [0,28]
  - 5 in [0,29]
  - 2 in [0,31]
  - 2 in [0,33]
  - 1 in [0,34]
  - 2 in [0,35]
  - 2 in [0,37]
  - 2 in [0,38]
  - 2 in [0,39]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)
Starting presolve at 0.00s

Starting pres

In [5]:
pack(container, boxes, 10)



Starting CP-SAT solver v9.8.3296
Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Parameters: log_search_progress: true
Setting number of workers to 16

Setting number of workers to 16

Initial satisfaction model '': (model_fingerprint: 0x26124a74348f784d)
#Variables: 48
  - 3 in [0,100]
  - 6 in [0,110]
  - 6 in [0,120]
  - 9 in [0,130]
  - 4 in [0,270]
  - 2 in [0,280]
  - 5 in [0,290]
  - 2 in [0,310]
  - 2 in [0,330]
  - 1 in [0,340]
  - 2 in [0,350]
  - 2 in [0,370]
  - 2 in [0,380]
  - 2 in [0,390]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Initial satisfaction model '': (model_fingerprint: 0x26124a74348f784d)
#Variables: 48
  - 3 in [0,100]
  - 6 in [0,110]
  - 6 in [0,120]
  - 9 in [0,130]
  - 4 in [0,270]
  - 2 in [0,280]
  - 5 in [0,290]
  - 2 in [0,310]
  - 2 in [0,330]
  - 1 in [0,340]
  - 2 in [0,350]
  - 2 in [0,370]
  - 2 in [0,380]
  - 2 in [0,390]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Starting pre

In [6]:
pack(container, boxes, 100)



Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Setting number of workers to 16
Setting number of workers to 16


Initial satisfaction model '': (model_fingerprint: 0x74eb0f3c80f0c54e)
#Variables: 48
  - 3 in [0,1000]
  - 6 in [0,1100]
  - 6 in [0,1200]
  - 9 in [0,1300]
  - 4 in [0,2700]
  - 2 in [0,2800]
  - 5 in [0,2900]
  - 2 in [0,3100]
  - 2 in [0,3300]
  - 1 in [0,3400]
  - 2 in [0,3500]
  - 2 in [0,3700]
  - 2 in [0,3800]
  - 2 in [0,3900]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Initial satisfaction model '': (model_fingerprint: 0x74eb0f3c80f0c54e)
#Variables: 48
  - 3 in [0,1000]
  - 6 in [0,1100]
  - 6 in [0,1200]
  - 9 in [0,1300]
  - 4 in [0,2700]
  - 2 in [0,2800]
  - 5 in [0,2900]
  - 2 in [0,3100]
  - 2 in [0,3300]
  - 1 in [0,3400]
  - 2 in [0,3500]
  - 2 in [0,3700]
  - 2 in [0,3800]
  - 2 in [0,3900]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#r

#1       1.08s probing_max_lp (fixed_bools=0/1664)#1       1.08s probing_max_lp (fixed_bools=0/1664)



Task timing                          n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
               'default_lp':         1 [   1.09s,    1.09s]    1.09s   0.00ns    1.09s         1 [   1.36s,    1.36s]    1.36s   0.00ns    1.36s
         'feasibility_pump':         1 [111.15us, 111.15us] 111.15us   0.00ns 111.15us         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
          'fj_long_default':         6 [ 87.89ms, 295.49ms] 180.38ms  60.56ms    1.08s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fj_long_lin_default':         6 [119.19ms, 227.08ms] 179.31ms  33.29ms    1.08s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
         'fj_short_default':         7 [  6.31ms, 178.55ms] 148.84ms  58.45ms    1.04s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
     'fj_short_lin_default

In [7]:
pack(container, boxes, 1000)



Starting CP-SAT solver v9.8.3296
Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Parameters: log_search_progress: true
Setting number of workers to 16
Setting number of workers to 16


Initial satisfaction model '': (model_fingerprint: 0x50f40e8ac0ad0a5b)
#Variables: 48
  - 3 in [0,10000]
  - 6 in [0,11000]
  - 6 in [0,12000]
  - 9 in [0,13000]
  - 4 in [0,27000]
  - 2 in [0,28000]
  - 5 in [0,29000]
  - 2 in [0,31000]
  - 2 in [0,33000]
  - 1 in [0,34000]
  - 2 in [0,35000]
  - 2 in [0,37000]
  - 2 in [0,38000]
  - 2 in [0,39000]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)
Initial satisfaction model '': (model_fingerprint: 0x50f40e8ac0ad0a5b)
#Variables: 48
  - 3 in [0,10000]
  - 6 in [0,11000]
  - 6 in [0,12000]
  - 9 in [0,13000]
  - 4 in [0,27000]
  - 2 in [0,28000]
  - 5 in [0,29000]
  - 2 in [0,31000]
  - 2 in [0,33000]
  - 1 in [0,34000]
  - 2 in [0,35000]
  - 2 in [0,37000]
  - 2 in [0,38000]
  - 2 in [0,39000]
#kInterval: 48
#kLinea

In [8]:
pack(container, boxes, 10_000)


Starting CP-SAT solver v9.8.3296

Parameters: log_search_progress: true
Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Setting number of workers to 16
Setting number of workers to 16


Initial satisfaction model '': (model_fingerprint: 0x758d3d7aad74b155)
#Variables: 48
  - 3 in [0,100000]
  - 6 in [0,110000]
  - 6 in [0,120000]
  - 9 in [0,130000]
  - 4 in [0,270000]
  - 2 in [0,280000]
  - 5 in [0,290000]
  - 2 in [0,310000]
  - 2 in [0,330000]
  - 1 in [0,340000]
  - 2 in [0,350000]
  - 2 in [0,370000]
  - 2 in [0,380000]
  - 2 in [0,390000]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Initial satisfaction model '': (model_fingerprint: 0x758d3d7aad74b155)
#Variables: 48
  - 3 in [0,100000]
  - 6 in [0,110000]
  - 6 in [0,120000]
  - 9 in [0,130000]
  - 4 in [0,270000]
  - 2 in [0,280000]
  - 5 in [0,290000]
  - 2 in [0,310000]
  - 2 in [0,330000]
  - 1 in [0,340000]
  - 2 in [0,350000]
  - 2 in [0,370000]
  - 2 in [0,380000]
  - 2 in [0,3

#1       0.37s probing_max_lp (fixed_bools=0/3616)
#1       0.37s probing_max_lp (fixed_bools=0/3616)
#2       0.38s quick_restart (fixed_bools=0/2211)
#2       0.38s quick_restart (fixed_bools=0/2211)


Task timing                          n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
               'default_lp':         1 [373.23ms, 373.23ms] 373.23ms   0.00ns 373.23ms         1 [ 51.22ms,  51.22ms]  51.22ms   0.00ns  51.22ms
         'feasibility_pump':         1 [ 88.72us,  88.72us]  88.72us   0.00ns  88.72us         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
          'fj_long_default':         2 [ 50.75ms, 321.99ms] 186.37ms 135.62ms 372.74ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fj_long_lin_default':         2 [119.85ms, 240.72ms] 180.28ms  60.44ms 360.56ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
         'fj_short_default':         3 [ 38.97ms, 168.77ms] 121.42ms  5